In [2]:
import pymssql
import pandas as pd
import numpy as np
from pandas import DataFrame

In [3]:
#connect to sinsinsa DB
conn = pymssql.connect(server="192.168.3.22:14321",user="psu",password="@psu!234@",database="SFKR_XDAS")
cursor=conn.cursor()

KeyboardInterrupt: 

# Error data collect

In [4]:
cursor.execute("select OTIME,VALUE from DAQ_L03A0 where (NAME like '%D009025') and (convert(varchar,OTIME,121) between '2020-10-13' and '2020-10-24') and (convert(varchar,OTIME,8) between '09:00:00' and '19:00:00') and VALUE=1 order by OTIME")
table_error=cursor.fetchall()
err_data=pd.DataFrame(data=table_error,columns=['OTIME','VALUE'])

In [6]:
err_data

,OTIME,VALUE
0,2020-10-13 09:40:11.453,1
1,2020-10-13 09:59:42.386,1
2,2020-10-13 11:03:51.092,1
3,2020-10-13 12:16:44.834,1
4,2020-10-13 13:30:46.580,1
...,...,...
191,2020-10-23 11:49:11.625,1
192,2020-10-23 14:13:52.173,1
193,2020-10-23 16:27:11.631,1
194,2020-10-23 16:28:27.255,1


# non Error data collect

In [7]:
cursor.execute("select OTIME,VALUE from DAQ_L03A0 where (NAME like '%D009025') and (convert(varchar,OTIME,121) between '2020-10-13' and '2020-10-24') and (convert(varchar,OTIME,8) between '09:00:00' and '19:00:00') and VALUE=0 order by OTIME")
table_non=cursor.fetchall()
nonerr_data=pd.DataFrame(data=table_non,columns=['OTIME','VALUE'])

In [8]:
nonerr_data

,OTIME,VALUE
0,2020-10-13 09:38:45.312,0
1,2020-10-13 09:57:36.144,0
2,2020-10-13 10:47:08.170,0
3,2020-10-13 11:16:19.460,0
4,2020-10-13 12:27:48.631,0
...,...,...
200,2020-10-23 11:55:36.542,0
201,2020-10-23 16:23:31.878,0
202,2020-10-23 16:27:19.422,0
203,2020-10-23 16:30:44.489,0


# 에러발생 전 2초간 데이터 수집

In [66]:
err_result_data=np.zeros(200,dtype=np.int64)
err_result_data.shape


(200,)

In [67]:
for row in range(0,len(err_data)):
    error_start=err_data['OTIME'][row]
    cursor.execute("select top 200 OTIME,VALUE from DAQ_L03A0 where (NAME like '%D902') and (OTIME between '2020-10-13 09:00:00' and '"+ error_start +"') order by OTIME desc")
    table_error=cursor.fetchall()
    temp_errDF=pd.DataFrame(data=table_error,columns=['OTIME','VALUE'])
    stack_data=temp_errDF['VALUE'].to_numpy(dtype=np.int64)
    err_result_data=np.vstack([err_result_data,stack_data])
            
    
            
            

In [68]:
err_result_data=np.delete(err_result_data, 0, axis = 0)


In [69]:
err_result_data

array([[ 558,  547,  533, ...,  121,  122,  121],
       [1385, 1401, 1415, ...,  956,  927,  923],
       [1012, 1015, 1025, ..., 1376, 1379, 1369],
       ...,
       [ 694,  689,  668, ...,  175,  176,  179],
       [1498, 1468, 1454, ...,  653,  656,  651],
       [1321, 1302, 1308, ...,  716,  731,  741]], dtype=int64)

# 정상발생 전  2초간 데이터 수집

In [54]:
nonerr_result_data=np.zeros(200,dtype=np.int64)

In [55]:
for row in range(0,len(nonerr_data)):
    nonerror_start=nonerr_data['OTIME'][row]
    cursor.execute("select top 200 OTIME,VALUE from DAQ_L03A0 where (NAME like '%D902') and (OTIME between '2020-10-13 09:00:00' and '"+ nonerror_start +"') order by OTIME desc")
    table_nonerror=cursor.fetchall()
    temp_nonerrDF=pd.DataFrame(data=table_nonerror,columns=['OTIME','VALUE'])
    stack_data=temp_nonerrDF['VALUE'].to_numpy(dtype=np.int64)
    nonerr_result_data=np.vstack([nonerr_result_data,stack_data])
            

In [56]:
for index in range(0,10):
    nonerr_result_data=np.delete(nonerr_result_data, index, axis = 0)

In [59]:
nonerr_result_data

array([[1078, 1085, 1086, ...,  857,  819,  796],
       [ 939,  950,  948, ...,  720,  705,  710],
       [1610, 1624, 1614, ..., 1198, 1205, 1230],
       ...,
       [1150, 1165, 1169, ...,  999, 1003, 1018],
       [ 937,  953,  958, ...,  910,  857,  860],
       [1292, 1285, 1301, ...,  740,  741,  750]], dtype=int64)

# 데이터 시프트

In [71]:
temp=np.roll(err_result_data,1)
for i in range(0,199):
    err_result_data=np.vstack([err_result_data,temp])
    temp=np.roll(temp,1)
    


In [72]:
err_result_data.shape

(39200, 200)

In [73]:
temp2=np.roll(nonerr_result_data,1)
for i in range(0,199):
    nonerr_result_data=np.vstack([nonerr_result_data,temp2])
    temp2=np.roll(temp,1)
    

In [75]:
nonerr_result_data.shape

(39200, 200)